In [68]:
# state = {'board': [1, 0, 2, 1, 0, 1, 3, 1, 0, 2, 1, 0], 'current_player': 1, 'player_territory': (0, 6)}
# action = 6
# play(state, action, True)

# New Game Play

In [57]:
import copy
import random 
from json.encoder import INFINITY

seed = 37

randint = random.randint
random.seed(seed)

In [36]:
def end_game(state):
    
    return sum(state['board']) == 0

In [37]:
def assign_reward(reward, new_reward):
    return [reward[0] + new_reward[0], reward[1] + new_reward[1], new_reward[2]]

In [38]:
def terminate_loop(state):
    state['board'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    reward = (0,0,0)

    return (state, reward)

In [39]:
def print_game_play(state, reward, new_starting_position):
    print('state: ', state)
    print('reward: ', reward)
    print('new_starting_position: ', new_starting_position)
    print('--------------------')

In [40]:
def four_left(state, reward):
    state['board'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    if reward[2] == 0: 
        return  (4,0,0)
    
    if reward[2] == 1: 
        return  (0,4,1)

In [41]:
def get_valid_actions(arr):
    new_arr = []
    for i,a in enumerate(arr):
        if a != 0:
            new_arr.append(i)
    return new_arr

In [66]:
def generate_action(state,agent):
      func = agent['func']
      arg = agent['arg']
      return func(state, arg)


In [43]:
def is_valid_actions(state):

    if state['current_player'] == 0:
          val = state['board'][0:state['player_territory'][1]]
          return sum(val)
    
    if state['current_player'] == 1:
          val = state['board'][state['player_territory'][1]:12]
          return sum(val)
    

In [71]:
a = 0
not a

True

In [44]:
def get_reward(stones_in_pit, board, position, stone, current_player, player_territory): 
    board[position] = 0
    if stone == stones_in_pit-1 and position < player_territory[1] and current_player == 1:
        return (0,4,1)

    if stone == stones_in_pit-1 and position >= player_territory[1] and current_player == 0:
        return  (4,0,0)

    if position < player_territory[1]:
        return  (4,0,0)
        # return (0,4,1)

    else:
        return (0,4,1)


In [45]:
def is_illegal_move(state, action):
    board = state['board']
    current_player = state['current_player']
    player_territory = state['player_territory']
    stones_in_pit = board[action]


    # is pit empty
    if stones_in_pit == 0:
        # print('pit is empty')
        return True
    
    # is pit not in player one's territory
    if current_player == 0 and action >= player_territory[1]:
        # print("pit is not in player 1's territory")
        return True
    
    # is pit not in player two's territory
    if current_player == 1 and action <  player_territory[1]:
        # print("pit is not in player 2's territory")
        return True
    
    return False


In [46]:
def session(state, starting_position,latest_winner ):
    new_state = copy.deepcopy(state)

    board = new_state['board']
    current_player = new_state['current_player']
    player_territory = new_state['player_territory']
    stones_in_pit = board[starting_position]
    board[starting_position] = 0
    reward = [0,0,latest_winner]


    for stone in range(stones_in_pit):
        future_position = (stone + starting_position + 1) % 12
        board[future_position] += 1

        if board[future_position] == 4:
             new_reward = get_reward(stones_in_pit, board, future_position, stone, current_player, player_territory)
             reward = assign_reward(reward, new_reward)

    return (new_state, reward, future_position)


In [58]:
def play(state, action, show=False):
    reward = [0,0,0]
    max_rez = 100
    rez = 0

    if is_illegal_move(state, action):
        state['board'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        return (state, reward)

    state, new_reward, new_starting_position = session(state, action, reward[2])
    reward = assign_reward(reward, new_reward)
    if show:
        print_game_play(state, reward, new_starting_position)

    board = state['board']
    stones_in_pit = board[new_starting_position]
    
    while stones_in_pit > 1:
        state, new_reward, new_starting_position = session(state,new_starting_position,reward[2])
        reward = assign_reward(reward, new_reward)
        if show:
            print_game_play(state, reward, new_starting_position)

        board = state['board']
        stones_in_pit = board[new_starting_position]

        if rez > max_rez:
            state, reward = terminate_loop(state)
            return (state, reward)

        rez += 1

    
    state['current_player'] = +(not state['current_player'])

    if sum(state['board']) <= 4:
        new_reward = four_left(state, reward)
        reward = assign_reward(reward, new_reward)

    if is_valid_actions(state) == -1:
        new_reward = terminate(state)
        state['current_player'] = +(not state['current_player'])
        reward = assign_reward(reward, new_reward)

    return (state, reward)

## Agents

In [67]:
def random_agent(state, arg=None):

    if state['current_player'] == 0:
        arr = state['board'][0:state['player_territory'][1]]
        valid_actions = get_valid_actions(arr)
        return valid_actions[randint(0,len(valid_actions)-1)]
    

    if state['current_player'] == 1:
        arr = state['board'][state['player_territory'][1]:12]
        valid_actions = get_valid_actions(arr)
        return valid_actions[randint(0,len(valid_actions)-1)] + state['player_territory'][1]

## Simulation

In [61]:
def simulate_game(state, player_1, player_2,show=False):
    state = copy.deepcopy(state)
    reward = [0,0,0]
    path = []

    while True:
        action = generate_action(state, player_1, player_2)

        if is_illegal_move(state, action):
            continue
        
        state, new_reward = play(state, action)
        reward = assign_reward(reward, new_reward)
        if show:
            print_game_play(state, reward, action)

        if end_game(state):
            break
        path.append(action)
    
    # print('state: ', state)
    return  (reward, path)

In [ ]:
# state = {
# 'board' :[4,4,4,4,4,4,4,4,4,4,4,4],
#   'current_player': 0,
#   'player_territory': (0,6)}

# simulate_game(state)

In [65]:
state = {
   'board' :[4,4,4,4,4,4,4,4,4,4,4,4],
   'current_player': 0,
   'player_territory': (0,6)
}
number_of_games = 10
lol = []

player_1 = {
   'func': random_player_1,
   'arg': {}
}

player_2 = {
   'func': random_player_2,
   'arg': {}
}

for j in range(2,11):
   state['player_territory'] = (0,j)
   print('player_territory',0,j)
   results = {
         'player_1_wins': 0,
         'player_2_wins': 0,
         'ties': 0,
      }
   for i in range(number_of_games):
      reward, path = simulate_game(state, player_1, player_2)
      lol.append([reward,path])
      if reward[0] > reward[1]:
         results['player_1_wins'] += 1 
      elif reward[0] < reward[1]:
         results['player_2_wins'] += 1 
      elif reward[0] == reward[1]:
         results['ties'] += 1 
    
   print(results)
   print('player_1_wins: ',results['player_1_wins']/number_of_games * 100)
   print('player_2_wins: ',results['player_2_wins']/number_of_games * 100)
   print('ties: ',results['ties']/number_of_games * 100)
   # print(lol)

player_territory 0 2


TypeError: generate_action() missing 2 required positional arguments: 'player_1' and 'player_2'

# Algorithms

## Minimax Algorithim

In [ ]:

def best_move(origin_game_state, max_dept=2):
    all_game_state = []
    all_game_state.append(origin_game_state)

    if origin_game_state['current_player'] == 0:
        best_score = -INFINITY
        for action in range(6):
            temp_state =  copy.deepcopy(all_game_state[-1])
            # temp_state['path'].append(child)
            game_state, new_reward = play(temp_state, action)
            reward = assign_reward(reward, new_reward)
            all_game_state.append(game_state)
            score = minimax(game_state, False, all_game_state, 2, max_dept, -INFINITY, INFINITY)
            all_game_state.pop()

            if score > best_score:
                best_score = score
                move = action

    else:
        best_score = INFINITY
        for child in range(6,12):
            temp_state =  copy.deepcopy(all_game_state[-1])
            temp_state['starting_position'] = child
            # temp_state['path'].append(child)
            ayo_board = temp_state['ayo_board']

            if ayo_board[child] != 0:
                game_state, new_reward = play_game(temp_state)
                all_game_state.append(game_state)
                score = minimax(game_state, True,all_game_state,1,max_dept,-INFINITY,INFINITY)
                all_game_state.pop()

                if score < best_score:
                    best_score = score
                    move = child
    print('best_score',best_score)
    return move



def minimax(game_state, is_maximizing,all_game_state,current_player,max_dept,alpha,beta ):
    max_dept -= 1
    game_state['current_player'] = current_player

    if(max_dept <= 0 or  end_game(game_state)):
        return game_state['player_1_points']-game_state['player_2_points']

    if is_maximizing:
        best_score = -INFINITY

        for action in range(6):
            temp_state =  copy.deepcopy(all_game_state[-1])
            temp_state['starting_position'] = action
            # temp_state['path'].append(child)
            game_state, new_reward = play(temp_state, action)
            all_game_state.append(game_state)
            score = minimax(game_state, False,all_game_state,2,max_dept,alpha,beta)
            all_game_state.pop()
            best_score = max(score, best_score)
            alpha = max(alpha,score)

            if beta <= alpha:
                break

        return best_score

    else:
        best_score = INFINITY

        for child in range(6,12):
            temp_state =  copy.deepcopy(all_game_state[-1])
            temp_state['starting_position'] = child
            # temp_state['path'].append(child)
            ayo_board = temp_state['ayo_board']

            if ayo_board[child] != 0:
                game_state, new_reward = play(temp_state)
                all_game_state.append(game_state)
                score = minimax(game_state, True,all_game_state,1,max_dept,alpha,beta)
                all_game_state.pop()
                best_score = min(score, best_score)
                beta = min(beta,score)

                if beta <= alpha:
                    break

        return best_score


In [ ]:
def simulate_game(state,player_1, player_2, show=False):
    state = copy.deepcopy(state)
    reward = [0,0,0]
    path = []

    while True:
        action = generate_action(state,player_1, player_2)

        if is_illegal_move(state, action):
            continue
        
        state, new_reward = play(state, action)
        reward = assign_reward(reward, new_reward)
        if show:
            print_game_play(state, reward, action)

        if end_game(state):
            break
        path.append(action)
    
    # print('state: ', state)
    return  (reward, path)

In [ ]:
state = {
   'board' :[4,4,4,4,4,4,4,4,4,4,4,4],
   'current_player': 0,
   'player_territory': (0,6)
}
number_of_games = 10000
lol = []


for j in range(2,11):
   state['player_territory'] = (0,j)
   print('player_territory',0,j)
   results = {
         'player_1_wins': 0,
         'player_2_wins': 0,
         'ties': 0,
      }
   for i in range(number_of_games):
      reward, path = simulate_game(state)
      lol.append([reward,path])
      if reward[0] > reward[1]:
         results['player_1_wins'] += 1 
      elif reward[0] < reward[1]:
         results['player_2_wins'] += 1 
      elif reward[0] == reward[1]:
         results['ties'] += 1 
    
   print(results)
   print('player_1_wins: ',results['player_1_wins']/number_of_games * 100)
   print('player_2_wins: ',results['player_2_wins']/number_of_games * 100)
   print('ties: ',results['ties']/number_of_games * 100)
   # print(lol)